In [21]:
import numpy as np
from PIL import Image

import random
import os
import json

In [22]:
WINDOW_SIZE = 512
WINDOW_THRESHOLD = 1500
WINDOWS_PER_SLICE = 40
SLICE_EVERY = 50
RAW_PATH = os.path.join('..', '2d unet full data', 'data', 'complete')
SAVE_PATH = os.path.join('data')

for stage in (['train', 'eval']):
    fault_raw_path = os.path.join(RAW_PATH, stage, 'fault')
    seis_raw_path = os.path.join(RAW_PATH, stage, 'seis')

    fault_save_path = os.path.join(SAVE_PATH, stage, 'fault')
    seis_save_path = os.path.join(SAVE_PATH, stage, 'seis')

    with open(os.path.join(RAW_PATH, stage, 'metadata.json'), 'r') as file:
        raw_metadata = json.load(file)

    img_cnt = 0
    metadata = []
    for i in range(len(raw_metadata)):
        if i % SLICE_EVERY == 0:
            record = raw_metadata[i]
            data = np.asarray(Image.open(os.path.join(seis_raw_path, record['data'])))
            labels = np.asarray(Image.open(os.path.join(fault_raw_path, record['label'])))
            timelines, xlines,  = np.shape(data)
            for _ in range(WINDOWS_PER_SLICE):
                while True:
                    random_xline = random.randint(0, xlines - WINDOW_SIZE)
                    random_timeline = random.randint(0, timelines - WINDOW_SIZE)
                    random_labels_window = labels[random_timeline:random_timeline+WINDOW_SIZE, random_xline:random_xline+WINDOW_SIZE]
                    if np.sum(random_labels_window) > WINDOW_THRESHOLD * 255:
                        break
                random_data_window = data[random_timeline:random_timeline+WINDOW_SIZE, random_xline:random_xline+WINDOW_SIZE]
                metadata.append({
                    'data': f'{img_cnt}.jpeg',
                    'label': f'{img_cnt}.jpeg'
                })
                data_img = Image.fromarray(random_data_window)
                label_img = Image.fromarray((random_labels_window))
                data_img.save(os.path.join(seis_save_path, f'{img_cnt}.jpeg'))
                label_img.save(os.path.join(fault_save_path, f'{img_cnt}.jpeg'))
                img_cnt += 1
    with open(os.path.join(SAVE_PATH, stage, 'metadata.json'), 'w') as file:
        json.dump(metadata, file)